# Technical Report: Raleigh Greenway Spatial Safety Audit
**Department:** Geospatial Analytics & Public Safety  
**Document Status:** Draft | Phase II: Exploratory Data Analysis  
**Reference System:** North Carolina State Plane (EPSG:2264)

---

## 1. Executive Summary
This notebook documents the exploratory analysis of police incident records filtered by spatial proximity to the City of Raleigh Greenway System. The objective is to identify high-density incident corridors to inform targeted safety visualizations in ArcGIS.

### 1.1 Data Provenance
| Dataset | Source | Format | Record Count |
| :--- | :--- | :--- | :--- |
| Municipal Incidents | Raleigh Police Dept | CSV/DuckDB | ~600,000 |
| Greenway Centerlines | City of Raleigh GIS | JSON/GeoJSON | 483 Segments |

---

## 2. Methodology & Geometric Constraints
The analysis employs a **Euclidean Buffer** approach to isolate relevant records. Given the variance in reporting accuracy and GPS signal integrity, the following mathematical constraint defines the primary analysis set $I_{g}$:

$$I_{g} = \{ p \in I \mid \min(\text{dist}(p, L)) \le 50\text{ft} \}$$

**Rationale:** The 50ft threshold is selected to account for the standard error of GPS transponders in wooded trail environments and the proximity of trail access points to roadway intersections.

In [1]:
# =============================================================================
# ENVIRONMENT SETUP & DATA INGESTION
# =============================================================================
import duckdb
import pandas as pd
import numpy as np

# Configure Pandas display options 
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

try:
    # Establish connection to the analytical database
    con = duckdb.connect('greenways.db')
    
    # Ingest the spatially joined analysis table
    # Standard SQL query to pull the 'baked' layer created in Phase I
    query = "SELECT * FROM analysis_greenway_incidents"
    df = con.execute(query).df()
    
    print(f"Data Pipeline Verified: {len(df)} records loaded.")
except Exception as e:
    print(f"Connection Error: {e}")

Data Pipeline Verified: 5214 records loaded.


In [2]:
# Preview the schema and initial records
df.head()

,greenway_name,object_id,global_id,case_number,crime_category,crime_description,district,reported_at,latitude,longitude,geom_point
0,Walnut Creek Trail,15143,166d8648-3ba6-4c9c-a161-aef606926d7c,P15051278,SEX OFFENSES,Sex Offense/Peeping Tom,Southwest,2015-09-17 02:52:00,35.77,-78.69,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,Little Rock Trail,30509,338f6523-c1d5-4e3d-ac9a-649170a7bf16,P18009197,FRAUD,Fraud/False Pretense or Swindle,Downtown,2018-02-19 22:28:00,35.77,-78.63,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,Little Rock Trail,30509,338f6523-c1d5-4e3d-ac9a-649170a7bf16,P18009197,FRAUD,Fraud/False Pretense or Swindle,Downtown,2018-02-19 22:28:00,35.77,-78.63,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,Reedy Creek Trail,57446,b37f9550-11c6-443f-a47c-a6206b56a85c,P17042175,VANDALISM,All Other/Damage to Property (minor),Northwest,2017-08-04 19:35:00,35.82,-78.71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,Reedy Creek Trail,57446,b37f9550-11c6-443f-a47c-a6206b56a85c,P17042175,VANDALISM,All Other/Damage to Property (minor),Northwest,2017-08-04 19:35:00,35.82,-78.71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
